In [35]:
import pandas as pd
import plotly.express as px

In [36]:
df_producto=pd.read_csv('../data/expo_doc_completo.csv', 
                        encoding='Latin-1',
                        # sep=';', 
            dtype={'ncm':'str','CUIT':'str','empresa':'str','enmienda':'str','ncm_descri':'str','sim':'str','pais':'str','pais_descri':'str','dia':'str','mes':'str','anio':'str','sec':str}
            )

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_16564\1921368730.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_producto=pd.read_csv('../data/expo_doc_completo.csv',


In [37]:
df_producto.cuit=df_producto.cuit.astype(str)

In [38]:
# Correcciones de lectura
df_producto.pais_descri=df_producto.pais_descri.apply(lambda x: x.capitalize()) #Mayuscula a los países
df_producto.empresa=df_producto.empresa.apply(lambda x: x.lower()) #Minuscula en los nombres de las empresas

# Fechas para usarlas en los graficos: Se usa más adelante
desde=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'][0]
hasta=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'].iloc[-1]

inicio=df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).mes[0]+'/01/'+df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).anio[0]
mes=int(df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).mes.iloc[-1])+1
fin=str(mes)+'/01/'+df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).anio.iloc[-1]
monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=inicio, end=fin, freq='M').strftime('%m-%Y')
})

In [39]:
top_empresas=pd.read_excel('../output/ex-derivados soja.xlsx',dtype={'cuit':str})
top_empresas.cuit=top_empresas.cuit.astype(str)

In [40]:
expo_desagregado=df_producto[(df_producto.empresa.isin(top_empresas.empresa))&(df_producto.anio>='2021')].reset_index(drop=True)
expo_top_empresas=expo_desagregado.groupby(['mes','anio'],as_index=False).sum()[['mes','anio','fob']]
expo_top_empresas=expo_top_empresas.sort_values(['anio','mes'],ascending=True).reset_index(drop=True)

In [41]:
expo_top_empresas.to_excel('../output/expo_top_empresas.xlsx',index=False)

# Excel
writer = pd.ExcelWriter('../output/expo_top_empresas.xlsx', engine='xlsxwriter')
expo_top_empresas.to_excel(writer, sheet_name='Expo indec', index=False)
expo_desagregado.to_excel(writer, sheet_name='Expo indec por empresa', index=False)
writer.save()